In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [ ]:
from exp import nb_resnet_ssd_std

In [ ]:
import sys 
sys.path.append("..") 
#from exp import nb_bbox_hw_statistics
from detect_symbol.exp import nb_databunch
from detect_symbol.exp import nb_init_model
from detect_symbol.exp import nb_resnet_ssd
from detect_symbol.exp import nb_anchors_loss_metrics

In [ ]:
# export
import torch

In [ ]:
# export
from torch.nn import functional as F

In [ ]:
# export
from torch import tensor

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# export
from IPython.core import debugger as idb

In [ ]:
# export
import numpy as np

In [ ]:
# export
import math

## functions

In [ ]:
def find_neibs(idx, grids = (49, 49), dis = 1):
    '''
    找到某个anchor周围相邻的anchors的下标里列表。距离默认1。
    这个任务中只有第一层的grids参与，所以只需要第一次的grids的尺寸。
    anchor也是1对1的。
    参数：
        idx：目标anchor在grid anchors(get_grid_anchors返回的gvs)列表中的下标
        grids: 尺寸
        dis：邻居的距离
    返回值：
        邻居的下标列表
    '''
    gh, gw = grids
    x = idx // gh
    y = idx % gw
    ret = []
    for nx in range(x - dis, x + dis + 1):
        for ny in range(y - dis, y + dis + 1):
            if nx >= 0 and ny >= 0 and nx < gw and ny < gh \
                    and not(nx == x and ny == y):
                nidx = ny * gw + nx
                #print(x, y, nx, ny, nidx)
                ret += [nidx]
    return ret      
                
    

In [ ]:
# export
#定义一个新的GridAnchor_Functions，主要是修改get_scroe_hits
class GridAnchor_Funcs_std(nb_anchors_loss_metrics.GridAnchor_Funcs):
    def __init__(self, fig_hw, grids, device):
        anchors = [[(1, 1)]]
        gvs,ghs,gws,avs,ahs,aws = nb_anchors_loss_metrics.get_grids_anchors( \
                    fig_hw, grids, anchors)
        self.grids = grids
        super().__init__(gvs, avs, device)
        
    def get_scores_hits(self, gt_bboxs): 
        # ground truch bbox center x,y
        gt_cx = gt_bboxs[:,[0,2]].mean(-1)
        gt_cy = gt_bboxs[:,[1,3]].mean(-1)

        # 判断目标bbox的中心落在哪个cell内
        hits = ((gt_cx[:,None] >= self.gvs[:,0][None]) &
                (gt_cx[:,None] <  self.gvs[:,2][None]) &
                (gt_cy[:,None] >= self.gvs[:,1][None]) &
                (gt_cy[:,None] <  self.gvs[:,3][None]))
        
        # 在的就是1，不在就是0
        scores = iou(gt_bboxs[:,None,:], fxdAncVs)
        
        return scores,hits

### zip as function

In [ ]:
# export
def get_ga1():
    return get_grids_anchors(fig_hw = (776,776), grids = [(49,49)]
                             , anchors = [[(1,1)]])

### total loss

In [ ]:
# export
def yolo_L_std(pred_batch, *gt_batch, conf_th=0.5,
           lambda_cent=1, lambda_pconf=1, lambda_nconf=1, lambda_clas=1, lambda_hw=1, clas_weights=None, gaf):
    '''
    与detect_symbol里面的yolo_L相比的区别是：
        不计算宽高方面的损失
        neg_idx要去掉find_neibs返回的discard列表
    
    
    clas_weights: 
    为了解决数据集的imbalance问题，一种方法是在dataloader中使用WeightedRandomSampler，但是这种方法不适用于目标检测问题。
    因为，（1）目标检测的label不是一个简单的数值（2）目标检测问题的一张图片可能包括不同类别的多个目标。
    所以为了解决目标检测问题中的imbalance问题，我们的方法是在损失函数中使用权重。
    为各类别分配权重，各目标对应的损失乘以该目标所属类别的权重。
    默认为None，即不使用权重。
    若设置非None，则clas_weights应该是一个一维tensor，其长度等于数据集的类别数。
    若设置为全1，则相当于不使用权重。
    合理的设置应保证所有元素之和等于数据集的类别数，否则相当于对损失函数的整体做了缩放。
    '''
    clas_loss = 0
    cent_loss = 0
    pconf_loss = 0
    nconf_loss = 0
    hw_loss = 0
    pos_cnt = 0
    neg_cnt = 0
    #import pdb; pdb.set_trace()
    for pred_txy,pred_conf,pred_clas,pred_thw,gt_bboxs,gt_clas in zip(*pred_batch, *gt_batch):
        keep = get_y(gt_bboxs)
        if keep.numel()==0: continue
          
        gt_bboxs = gt_bboxs[keep]
        gt_clas = gt_clas[keep]
        
        gt_bboxs = (gt_bboxs + 1) / 2
        gt_clas = gt_clas - 1 # the databunch add a 'background' class to classes[0], but we don't want that,so gt_clas-1
        
        if clas_weights is not None: ws = clas_weights[gt_clas]
        else: ws = None
        
        scores,hits = gaf.get_scores_hits(gt_bboxs)
        idx = idx_fromScoresHits(scores,hits)
        
        # classification loss
        pred_clas = pred_clas[idx]
        clas_loss += F.cross_entropy(pred_clas, gt_clas, weight=clas_weights, reduction='sum')
        
        # bbox center loss
        gt_t = gaf.b2t(gt_bboxs,idx,eps=1)
        pred_txy = pred_txy[idx]
        if ws is not None:
            cent_loss += ((gt_t[...,:2]-pred_txy)*ws[...,None]).abs().sum()
        else:
            cent_loss += (gt_t[...,:2]-pred_txy).abs().sum()
        
        # bbox height and width loss
        '''
        pred_thw = pred_thw[idx]
        if ws is not None: 
            hw_loss += ((gt_t[...,2:]-pred_thw)*ws[...,None]).abs().sum()
        else: 
            hw_loss += (gt_t[...,2:]-pred_thw).abs().sum()
        '''
        
        # positive confidence loss
        conf_pos = pred_conf[idx]
        if ws is not None: 
            pconf_loss += F.binary_cross_entropy_with_logits(conf_pos,torch.ones_like(conf_pos),weight=ws[...,None],reduction='sum')
        else: 
            pconf_loss += F.binary_cross_entropy_with_logits(conf_pos,torch.ones_like(conf_pos),reduction='sum')
        
        # negative conficence loss
        scores[range(0,len(idx)),idx] = conf_th + 1 # 强制责任anchor的得分好于threshold
        tmp = scores>conf_th # 判断各得分是否好于threshold
        tmp = tmp.max(dim=0)[0] # 各anchor是否有任意一个好于threshold的得分
        neg_idx = torch.where(tmp==0)[0] # 如果没有，该anchor是negative anchor
        conf_neg = pred_conf[negn_idx]
        nconf_loss += F.binary_cross_entropy_with_logits(conf_neg,torch.zeros_like(conf_neg),reduction='sum')
        
        pos_cnt += len(idx)
        neg_cnt += len(neg_idx)
        
    clas_loss  = lambda_clas  * clas_loss  /pos_cnt
    cent_loss  = lambda_cent  * cent_loss  /pos_cnt
    pconf_loss = lambda_pconf * pconf_loss /pos_cnt
    nconf_loss = lambda_nconf * nconf_loss /neg_cnt
    ###hw_loss    = lambda_hw    * hw_loss   /pos_cnt
    
    return clas_loss + cent_loss + pconf_loss + nconf_loss### + hw_loss

## test

In [ ]:
#get_grid_anchors的生成顺序
grids = [(49,49),(25,25),(13,13)]
anchors = [[(22,17),(22,37),(43,17),(43,37)],
            [(43,77),(83,37),(83,77)],
            [(83,162),(162,77),(162,162)]]
t = [(x, y, gx, gy, ax, ay) for (gx,gy),ancs in zip(grids,anchors)
                                for y in range(gy)
                                for x in range(gx)
                                for ax,ay in ancs]
print(t[:100])

In [ ]:
#测试find_neibs
neibs = find_neibs(100, (49, 49))

gvs,ghs,gws,avs,ahs,aws = nb_anchors_loss_metrics.get_grids_anchors(fig_hw \
                                , grids, anchors)
print('邻居', neibs)
for idx in neibs:
    print(idx, anchors)
    print(avs[idx])

In [ ]:
#距离之间的事情可以不考虑？先按照resnet18来一个
# 获取 grids 和 anchors
gvs,ghs,gws,avs,ahs,aws = nb_anchors_loss_metrics.get_grids_anchors(
                                            fig_hw = (776,776),
                                            grids = [(49,49)],
                                            anchors = [[(1,1)]])

In [ ]:
# fig_hw = (784,784)
# grids = [(49,49),(25,25),(13,13)]
# anchors = [[(22,17),(22,37),(43,17),(43,37)],
#                                          [(43,77),(83,37),(83,77)],
#                                          [(83,162),(162,77),(162,162)]]
#for (gx,gy),ancs in zip(grids,anchors) : 
#        print(gx,gy,ancs)
        
#at = [[gx, gy, ancs, y, x]  for (gx,gy),ancs in zip(grids,anchors) for y in range(gy) for x in range(gx)]    
#print(at)

In [ ]:
# 这些变量的尺寸
gvs.shape, ghs.shape, gws.shape, avs.shape, ahs.shape, aws.shape

In [ ]:
49*49*1  #只有一个anchor

In [ ]:
device = torch.device('cuda', 0)

In [ ]:
#ds = './data/ds_20200429'
#data = nb_databunch.get_databunch(data_root=ds, bs=4, device=device, cache=False) 
# get databunch
data = nb_databunch.get_databunch(data_root = '../detect_symbol/data/ds_20200429', bs=4,device=device)
x,y = data.one_batch()

In [ ]:
# get model and load init parameters

model = nb_resnet_ssd_std.get_resnet18_1ssd_std()
#model.load_state_dict(torch.load('./models/resnet18_ssd_init.pth'));

In [ ]:
# check the statistics of layer outputs
#nb_init_model.show_layer_stats(model,x)

In [ ]:
# pred a batch
pred = model(x)

### find idx of the responsible anchor

In [ ]:
# 选择一个bbox
samp = 1 # samp: sample, get one bbox from this sample
bb = (y[0][samp][-1]+1)/2
bb

In [ ]:
gaf = GridAnchor_Funcs_std(fig_hw = (776,776)
                         , grids = [(49,49)]
                         , device = device)

In [ ]:
# 测试 get_scores_hits 和 idx_fromScoresHits
scores,hits= gaf.get_scores_hits(bb[None])
idx = idx_fromScoresHits(scores,hits)
idx

In [ ]:
# 用另一种方法与之对照验证
test_idx = test_getIdx(bb.clone())
test_idx

In [ ]:
assert idx.item()==test_idx.item()

### losses

In [ ]:
# get GridAnchor_Funcs object
gvs,_,_,avs,_,_ = get_ga433()
gaf = GridAnchor_Funcs(gvs,avs,torch.device('cpu'))

In [ ]:
# 设置一个简单的clas_weights，你可以更改它的值来试验
clas_weights = torch.ones(16)*2

In [ ]:
# classification loss
clas_loss = clas_L(pred,*y,gaf=gaf, clas_weights=clas_weights)
clas_loss

In [ ]:
# bbox center loss
cent_loss = cent_L(pred,*y,gaf=gaf, clas_weights=clas_weights)
cent_loss

In [ ]:
# positive confidence loss
pconf_loss = pConf_L(pred,*y,gaf=gaf, clas_weights=clas_weights)
pconf_loss

In [ ]:
# negative confidence loss
nconf_loss = nConf_L(pred,*y,gaf=gaf)
nconf_loss

In [ ]:
# bbox height and width loss
hw_loss = hw_L(pred,*y,gaf=gaf, clas_weights=clas_weights)
hw_loss

In [ ]:
# total loss
total_loss = yolo_L(pred,*y, clas_weights=clas_weights,gaf=gaf)
total_loss

In [ ]:
assert (clas_loss+cent_loss+pconf_loss+hw_loss+nconf_loss)==total_loss

## test

### classification accuracy

In [ ]:
clas_acc(pred,*y,gaf=gaf), 

In [ ]:
n_clas = len(data.train_ds.y.classes)-1
print(1/n_clas)

### bbox center difference

In [ ]:
cent_d(pred,*y,gaf=gaf)

In [ ]:
((0.5/49)*49*49+(0.5/25)*25*25+(0.5/13)*13*13)/(49*49+25*25+13*13)

### bbox height and width ratio

In [ ]:
hw_r(pred,*y,gaf=gaf)

这个值是否合理？ 

下面我仅考虑高度 h 来分析，宽度 w 与之是同样的情况。  
  
根据定义，$b_h=p_h\cdot e^{t_h}$，即 $log(\frac{b_h}{p_h})=t_h$，其中 $t_h$ 是初始化模型的输出，属于标准正态分布。  
  
其中 $p_h$ 是 anchor 高度，因为我们在设计 anchor 时选择了 $p_h$ 位于 $o_h$（表示anchor所负责的那部分目标框的宽度）的分布的中心（对数坐标下），即$log(\frac{o_h}{p_h})$是均值为0的随机分布，我们假设这个随机分布也是正态分布，标准差为$\sigma_o$（小于1，因为我们设计 anchor 时，它所负责的目标框的宽度的范围约为\[-1.4,+1.4\]（对数坐标），那么可以粗略估计其标准差在0.5左右）。从正太分布的对称性上看，$log(\frac{p_h}{o_h})=-log(\frac{o_h}{p_h})$属于同样的分布。

则$log(\frac{b_h}{o_h})=log(\frac{b_h}{p_h})+log(\frac{p_h}{o_h})$应是均值为0，标准差为$\sqrt{1+\sigma_o^2}\approx\sqrt{1.25}\approx1.1$的正态分布。

我们令 $a=log(\frac{b_h}{o_h})$ 属于 $N(0,1.1)$，下面生成随机数

In [ ]:
a = torch.normal(0,1.1,size=(1,1000)).squeeze()
a.mean(), a.std()

对照代码，我们生成的 a 对应 `(pred_hw/gt_hw).log()` 部分，所以

In [ ]:
logR = a.abs().sum()
logR

因为我们这里仅考虑了 h，所以相当于 `cnt` 不需要 `*2`

In [ ]:
math.exp(logR/1000)

得到的值与调用`hw_r()`的结果是接近的。

# export

In [ ]:
!python notebook2script.py --fname 'anchors_loss_metrics.ipynb' --outputDir './exp/'